In [1]:
import os
import pandas as pd
from lets_plot import *
LetsPlot.setup_html()

In [2]:
merge_file = [os.path.join('../data/waitrose',file) for file in os.listdir('../data/waitrose') if file.endswith('.csv')]

In [3]:
df = pd.concat((pd.read_csv(file) for file in merge_file))


In [21]:
df= df.drop_duplicates()
df = df.rename(columns={
    'data-product-id': 'id',
    'data-product-type':'type',
    'data-product-name':'name',
    "data-product-on-offer":'offer',
    'product-size':'size'})    

,id,name,type,offer,data-product-index,image-url,product-page,product-name,size,item-price,price-per-unit,offer-description,category
0,525635,Organix Raspberry & Apple Soft Oaty Bars,G,False,1.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Raspberry & Apple Soft Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent"
1,557746,Organix Carrot Cake Oaty Bars,G,False,2.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Carrot Cake Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent"
2,32062,Aptamil 2 Follow On Milk,G,False,394.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/aptamil...,Aptamil 2 Follow On Milk,800g,13.50,£16.88/kg,NaN,"Baby, Child & Parent"
3,767801,Essential Baby Wipes,G,False,4.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Baby Wipes,80s,0.95,1.2p each,NaN,"Baby, Child & Parent"
4,514054,Organix Apple Rice Cakes,G,False,5.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Apple Rice Cakes,40g,1.60,£4/100g,NaN,"Baby, Child & Parent"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1589,18427,Duchy Organic British Lamb Half Leg,G,False,1590.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/duchy-o...,Duchy Organic British Lamb Half Leg,Typical weight 0.94kg,24.31,£26.00/kg,NaN,Waitrose Brands
1590,11606,Cooks' Ingredients Wok Oil,G,False,1591.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/cooks-i...,Cooks' Ingredients Wok Oil,250ml,2.40,96p/100ml,NaN,Waitrose Brands
1591,6903,No. 1 Badoz Vacherin Du Haut-Doubs AOP French ...,G,False,1592.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/no-1-ba...,No. 1 Badoz Vacherin Du Haut-Doubs AOP French ...,350g,10.00,£28.58/kg,NaN,Waitrose Brands
1592,6125,Cooks' Ingredients White Marzipan,G,False,1593.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/cooks-i...,Cooks' Ingredients White Marzipan,500g,2.50,£5/kg,NaN,Waitrose Brands


## price cleaning


In [5]:
def clean_item_price(item_price: str):  
    if 'each est.' in item_price:
        item_price = item_price.replace('each est.','')
    if type(item_price) !=str:
        pass
    elif '£' in item_price:
        item_price = item_price.replace('£','')
    elif 'p' in item_price :
        item_price = item_price.replace('p','')
        item_price = '0.'+item_price
    
    if '-' in item_price:
        item_price=item_price.split('-')[0]
    return float(item_price)

In [6]:
df['item-price'] = df['item-price'].astype('str').apply(clean_item_price)

## size cleaning

In [7]:
selected_rows = df['item-price'].astype(str).str.contains('-')

df.loc[selected_rows,'size'] = df.loc[selected_rows,'size'].str.split('-').str[0].str.replace('(','')


In [8]:
size_red = ['offer','category']
df[size_red]= df[size_red].astype('category')

df['id']=df['id'].astype('int32')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25378 entries, 0 to 1593
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   id                  25378 non-null  int32   
 1   name                25378 non-null  object  
 2   type                25378 non-null  object  
 3   offer               25378 non-null  category
 4   data-product-index  25368 non-null  float64 
 5   image-url           25378 non-null  object  
 6   product-page        25378 non-null  object  
 7   product-name        25367 non-null  object  
 8   size                25323 non-null  object  
 9   item-price          25367 non-null  float64 
 10  price-per-unit      24936 non-null  object  
 11  offer-description   7178 non-null   object  
 12  category            25378 non-null  category
dtypes: category(2), float64(2), int32(1), object(8)
memory usage: 2.3+ MB


# TASK 01 – DATA CLEANING:

## data counts are not matching so we should clean the data

In [10]:
df['name'].unique()

array(['Organix Raspberry & Apple Soft Oaty Bars',
       'Organix Carrot Cake Oaty Bars', 'Aptamil 2 Follow On Milk', ...,
       "Cooks' Ingredients Frozen Herby Chimichurri Portions",
       "Cooks' Ingredients Artichokes in Oil",
       "Cooks' Ingredients Frozen Stir Fry Vegetable Mix"], dtype=object)

In [11]:
df_cat = (
    df.groupby('id',as_index=False).agg({
        'id':'first',
        'category' : lambda x: list(x),
        'item-price':'first'
    })
)

df_cat

,id,category,item-price
0,1023,"[Tea, Coffee & Soft Drinks]",7.10
1,1029,[Food Cupboard],2.30
2,1048,"[Dietary & Lifestyle, Food Cupboard]",2.75
3,1051,"[Dietary & Lifestyle, Food Cupboard]",5.00
4,1052,"[Dietary & Lifestyle, Food Cupboard]",2.75
...,...,...,...
16050,998674,"[New, Tea, Coffee & Soft Drinks]",3.25
16051,999031,"[Fresh & Chilled, New]",0.90
16052,999167,"[Food Cupboard, New]",1.50
16053,999432,"[Household, New]",1.00


In [12]:
df_wait = pd.merge(df,df_cat,on='id',how='left')

df_wait.sort_values('id').tail()

,id,name,type,offer,data-product-index,image-url,product-page,product-name,size,item-price_x,price-per-unit,offer-description,category_x,category_y,item-price_y
16677,999167,Merchant Gourmet Coconut & Lime Leaf Wholegrai...,G,True,86.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/merchan...,Merchant Gourmet Coconut & Lime Leaf Wholegrai...,250g,1.5,£6/kg,Introductory Offer.Will be £2.00,New,"[Food Cupboard, New]",1.5
16219,999432,Andrex Fresh & Sensitive Wipes,G,False,557.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/andrex-...,Andrex Fresh & Sensitive Wipes,30Sheet,1.0,£3.34/100 sheets,NaN,Household,"[Household, New]",1.0
16840,999432,Andrex Fresh & Sensitive Wipes,G,False,249.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/andrex-...,Andrex Fresh & Sensitive Wipes,30Sheet,1.0,£3.34/100 sheets,NaN,New,"[Household, New]",1.0
16839,999942,Zebra Z-Grip Pack of 8 Assorted Ballpoint Pen,G,False,248.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/zebra-z...,Zebra Z-Grip Pack of 8 Assorted Ballpoint Pen,8Each,4.0,50p each,NaN,New,"[Home, New]",4.0
14730,999942,Zebra Z-Grip Pack of 8 Assorted Ballpoint Pen,G,False,141.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/zebra-z...,Zebra Z-Grip Pack of 8 Assorted Ballpoint Pen,8Each,4.0,50p each,NaN,Home,"[Home, New]",4.0


In [13]:
df_wait.drop(columns=[
    'item-price_y'
],inplace=True)

In [14]:
df_wait = df_wait.rename(
    columns={
        'item-price_x':'item-price',
        'category_x':'category',
        'category_y':'categories'
    }
)

In [15]:
df_wait.head()

,id,name,type,offer,data-product-index,image-url,product-page,product-name,size,item-price,price-per-unit,offer-description,category,categories
0,525635,Organix Raspberry & Apple Soft Oaty Bars,G,False,1.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Raspberry & Apple Soft Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent","[Baby, Child & Parent]"
1,557746,Organix Carrot Cake Oaty Bars,G,False,2.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Carrot Cake Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent","[Baby, Child & Parent]"
2,32062,Aptamil 2 Follow On Milk,G,False,394.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/aptamil...,Aptamil 2 Follow On Milk,800g,13.50,£16.88/kg,NaN,"Baby, Child & Parent","[Baby, Child & Parent]"
3,767801,Essential Baby Wipes,G,False,4.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Baby Wipes,80s,0.95,1.2p each,NaN,"Baby, Child & Parent","[Baby, Child & Parent, Summer, Toiletries, Hea..."
4,514054,Organix Apple Rice Cakes,G,False,5.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Apple Rice Cakes,40g,1.60,£4/100g,NaN,"Baby, Child & Parent","[Baby, Child & Parent, Dietary & Lifestyle]"


## TASK 02 – COMPARATIVE ANALYSIS:

In [16]:
everyday = df_wait[df_wait['category'].str.contains('Everyday value',na=False,case=False)].head()

branded = df_wait[~df_wait['category'].str.contains('Everyday value',na=False,case=False)].head(5)

branded

,id,name,type,offer,data-product-index,image-url,product-page,product-name,size,item-price,price-per-unit,offer-description,category,categories
0,525635,Organix Raspberry & Apple Soft Oaty Bars,G,False,1.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Raspberry & Apple Soft Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent","[Baby, Child & Parent]"
1,557746,Organix Carrot Cake Oaty Bars,G,False,2.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Carrot Cake Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent","[Baby, Child & Parent]"
2,32062,Aptamil 2 Follow On Milk,G,False,394.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/aptamil...,Aptamil 2 Follow On Milk,800g,13.50,£16.88/kg,NaN,"Baby, Child & Parent","[Baby, Child & Parent]"
3,767801,Essential Baby Wipes,G,False,4.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Baby Wipes,80s,0.95,1.2p each,NaN,"Baby, Child & Parent","[Baby, Child & Parent, Summer, Toiletries, Hea..."
4,514054,Organix Apple Rice Cakes,G,False,5.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Apple Rice Cakes,40g,1.60,£4/100g,NaN,"Baby, Child & Parent","[Baby, Child & Parent, Dietary & Lifestyle]"


In [17]:
df_wait = pd.concat([everyday,branded],ignore_index=True)

ddd = df_wait[['id','name','categories','item-price','category']]
ddd

,id,name,categories,item-price,category
0,86468,Essential Cucumber,"[Beer, Wine & Spirits, Dietary & Lifestyle, Ev...",0.95,Everyday Value
1,86412,Essential Red Peppers,"[Dietary & Lifestyle, Everyday Value, Fresh & ...",0.60,Everyday Value
2,88528,Essential Fairtrade Bananas,"[Dietary & Lifestyle, Everyday Value, Fresh & ...",0.95,Everyday Value
3,85115,Essential Onions,"[Dietary & Lifestyle, Everyday Value, Fresh & ...",0.15,Everyday Value
4,88411,Essential Lemons,"[Beer, Wine & Spirits, Dietary & Lifestyle, Ev...",0.35,Everyday Value
5,525635,Organix Raspberry & Apple Soft Oaty Bars,"[Baby, Child & Parent]",3.15,"Baby, Child & Parent"
6,557746,Organix Carrot Cake Oaty Bars,"[Baby, Child & Parent]",3.15,"Baby, Child & Parent"
7,32062,Aptamil 2 Follow On Milk,"[Baby, Child & Parent]",13.50,"Baby, Child & Parent"
8,767801,Essential Baby Wipes,"[Baby, Child & Parent, Summer, Toiletries, Hea...",0.95,"Baby, Child & Parent"
9,514054,Organix Apple Rice Cakes,"[Baby, Child & Parent, Dietary & Lifestyle]",1.60,"Baby, Child & Parent"


In [18]:
(
    ggplot(ddd,aes(x='name',y='item-price',color = 'category'))+
    # geom_bar(stat='identity')+
    # geom_smooth(method = "lm") +
    geom_line() +
    geom_point()+
    ggsize(900, 600)+
    theme_minimal()+
    # scale_color_viridis() +
    theme(axis_text_x=element_text(angle=45, hjust=1))+
         labs(
        title="Item Price Comparison: Everyday Value vs Branded ",
        x="Product Name",
        y="Item Price",
        color="Category"
    ) 
)

In [19]:
(
    ggplot(ddd,aes(x='name',y='item-price',fill= 'category'))+
    geom_bar(
        stat='identity'
        # tooltips=layer_tooltips()
        #     .line('@category')  
        #     .line('@name')  
        #     .line('@item-price')
            )+
    ggsize(900, 600)+
    theme_minimal()
    # theme(axis_text_x=element_blank()) 
)